<a href="https://colab.research.google.com/github/Leksandretta/inno_arch/blob/main/%D0%9F%D1%80%D0%BE%D0%BC%D0%B5%D0%B6%D1%83%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%B0%D1%82%D1%82%D0%B5%D1%81%D1%82%D0%B0%D1%86%D0%B8%D1%8F_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import torch
from transformers import AutoTokenizer, AutoModel

In [63]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [64]:
df = pd.read_json('/content/RuBQ_2.0_test.json')
df.head(3)

,uid,question,query,answer_text,question_uris,question_props,answers,paragraphs_uids,tags,RuBQ_version,question_eng
0,0,Что может вызвать цунами?,SELECT ?answer \nWHERE {\n wd:Q8070 wdt:P828 ...,Землетрясение,[http://www.wikidata.org/entity/Q8070],[wdt:P828],"[{'type': 'uri', 'label': 'землетрясение', 'va...","{'with_answer': [35622], 'all_related': [35622...",[1-hop],1,What can cause a tsunami?
1,1,Кто написал роман «Хижина дяди Тома»?,SELECT ?answer \nWHERE {\n wd:Q2222 wdt:P50 ?...,Г. Бичер-Стоу,[http://www.wikidata.org/entity/Q2222],[wdt:P50],"[{'type': 'uri', 'label': 'Гарриет Бичер-Стоу'...","{'with_answer': [35652], 'all_related': [35652...",[1-hop],1,"Who wrote the novel ""uncle Tom's Cabin""?"
2,2,Кто автор пьесы «Ромео и Джульетта»?,SELECT ?answer \nWHERE {\n wd:Q83186 wdt:P50 ...,Шекспир,[http://www.wikidata.org/entity/Q83186],[wdt:P50],"[{'type': 'uri', 'label': 'Уильям Шекспир', 'v...","{'with_answer': [35676, 35677], 'all_related':...",[1-hop],1,"Who is the author of the play ""Romeo and Juliet""?"


In [65]:
df = df[['question', 'answer_text']]

In [66]:
df.sample(5)

,question,answer_text
866,В какой области находятся Соловецкие острова?,Архангельская
148,Кто создал Лестерский кодекс - тетрадь научных...,Леонардо да Винчи
47,Какой город на черноморском побережье называлс...,Феодосия
1642,Какой город является столицей Казахстана?,Нур-Султан
2098,Сколько лет продолжалась Столетняя война?,116


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2330 entries, 0 to 2329
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   question     2330 non-null   object
 1   answer_text  2330 non-null   object
dtypes: object(2)
memory usage: 36.5+ KB


In [68]:
stop_words = stopwords.words('russian') # стоп-слова
stemmer = SnowballStemmer(language='russian') # привести к общему виду
nltk_tokenizer = RegexpTokenizer(r'\w+')

In [69]:
cache_dict = {}

def stem(word):

    if word in cache_dict:
        return cache_dict[word]
    norm_form = stemmer.stem(word) # приводит слово к общему виду
    cache_dict[word] = norm_form
    return norm_form

def text_preprocessing(text, w2v=False):
    text = text.lower()
    text = re.sub('^\d*\n*|\n*\d*$', '', text) # убрать служебные символы
    tokens = nltk_tokenizer.tokenize(text)
    tokens = [token for token in tokens if not token in stop_words] # убрать стоп-слова
    # print(tokens)
    tokens = [stem(token) for token in tokens]
    return ' '.join(tokens)

In [70]:
df['stem_question'] = df.question.apply(text_preprocessing)

In [72]:
print(df.loc[50, 'question'])
print(df.loc[50, 'stem_question'])

Какой город является столицей Нигерии?
город явля столиц нигер


In [73]:
tfidf_vectorizer = TfidfVectorizer()
db_question_vectors = tfidf_vectorizer.fit_transform(df['stem_question'])
db_question_vectors.shape

(2330, 4283)

In [85]:
def answer_question(question):
    stemmed_question = text_preprocessing(question) # стоп-слова, стемминг вопроса
    question_vector = tfidf_vectorizer.transform([stemmed_question]) # векторизация вопроса
    cosine_sims = cosine_similarity(question_vector, db_question_vectors) # посчитать косинусные расстояния
    cosine_idx = cosine_sims.argmax() #  взять ближайший

    print(f'Closest question: {df.question[cosine_idx]}')
    return df.answer_text[cosine_idx]

In [86]:
user_question = df.loc[35, 'question']
answer = df.loc[35, 'answer_text']
print(f'Question: {user_question}\nAnswer: {answer}')

Question: В каком штате США расположена самая длинная пещера в мире (530 км) с забавным названием Флинт-Мамонтова?
Answer: Кентукки


In [88]:
print(f'Answer: {answer_question("Кто является матерью Владимира Мономаха?")}')

Closest question: Кто является матерью Владимира Мономаха?
Answer: Мономахиня
